# <center>GluonCVによる姿勢推定のデモ<center>

#<center>☆★動画を変更する場合、2.2項でパスとファイル名を変更する必要があります。★☆<center>

#0. Google Driveをマウント

#1. 必要なライブラリのインストール

##1.1. GluonCVのインストール (pip)

In [ ]:
!pip install mxnet gluoncv

In [ ]:
!pip install opencv-python==4.1.2.30
!pip install opencv-contrib-python==4.1.2.30

##1.3 進行状況を表示するtqdmと呼ばれるライブラリをダウンロードする。

In [ ]:
!pip install tqdm #進行状況を表示するライブラリ #https://blog2.cct-inc.co.jp/blog/python/python_progressbar/



---



#2. コードを実行

##2.1. フォルダ名を変数にするため、現在の時刻を取得する。

In [ ]:
import datetime
from pytz import timezone
dates=datetime.datetime.now(timezone('Asia/Tokyo'))
exept_microsec=dates.strftime("%Y-%m-%d-%H-%M")
print('[Today : %s]' % str(exept_microsec))

DATATIME=exept_microsec

[Today : 2020-09-17-15-35]


##2.1. フォルダ名やファイル名を変数にするため、現在の時刻を取得する。

<例>

mayo.jpeg

↓

mayo_2020年9月1日〇〇時XX分.jpeg

In [ ]:
#-----------------------------------------------------------------------------------------------------------
#変更箇所
#------------------------------------------------------------------------------------------------------------
INPUT_PATH="/content/drive/My Drive/datasets/" #入力動画までのパス名
#INPUT_FILE="videoplayback_short.mp4" #入力動画のファイル名
INPUT_FILE="C0003.MP4" #入力動画のファイル名

OUTPUT_PATH="/content/drive/My Drive/datasets/" + "PoseEstim_GluonCV_%s/" % DATATIME #出力動画のパス名 (exは「datasets」の中に「PoseEstim_GluonCV_日付」のフォルダを作成)
#------------------------------------------------------------------------------------------------------------
OUTPUT_FILE='PoseEstim_%s_from_%s' % (DATATIME, INPUT_FILE) #ex) 「PoseEstim_日付_from_videoplayback_short.mp4」

print("INPUT_PATH：", INPUT_PATH)
print("INPUT_FILE：", INPUT_FILE)
print("OUTPUT_PATH：", OUTPUT_PATH)
print("OUTPUT_FILE：", OUTPUT_FILE)

INPUT_PATH： /content/drive/My Drive/datasets/
INPUT_FILE： C0003.MP4
OUTPUT_PATH： /content/drive/My Drive/datasets/PoseEstim_GluonCV_2020-09-17-15-35/
OUTPUT_FILE： PoseEstim_2020-09-17-15-35_from_C0003.MP4


##2.3. 出力結果を保存するフォルダを作成する。

In [ ]:
import os
os.mkdir(OUTPUT_PATH)

##2.4. 入力動画ファイルを取得する。

In [ ]:
import base64
import io
from IPython.display import HTML

def play_movie(mp4_path):
  video = io.open(mp4_path, 'r+b').read()
  encoded = base64.b64encode(video)
  return HTML(data='''<video width="640" height="480" loop="true" autoplay="true" controls> 
                      <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                      </video>'''.format(encoded.decode('utf-8')))  # https://developer.mozilla.org/ja/docs/Web/HTML/Element/video

print("入力画像 : ",  INPUT_PATH+INPUT_FILE)

#play_movie(INPUT_PATH+INPUT_FILE)

入力画像 :  /content/drive/My Drive/datasets/C0003.MP4


In [ ]:
import os
import cv2

if os.path.isfile(INPUT_PATH+INPUT_FILE) ==True:
  print("パス「%s」において、\nファイル名「%s」の存在が確認できました。\n" % (INPUT_PATH, INPUT_FILE))
  print("動画の基礎データを取得します... ")
  print()
else:
  print("パス「%s」において、\nファイル名「%s」の存在が確認できません。パスを見直してください。" % (INPUT_PATH, INPUT_FILE))

cap = cv2.VideoCapture(INPUT_PATH+INPUT_FILE)
#time.sleep(1)  ### letting the camera autofocus
print(cap)

# encoder(for mp4)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
# output file name, encoder, fps, size(fit to image size)
width= cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)

axes = None
num_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

print("fps     : ", fps)
print("width  : ", width)
print("height : ", height)
print("frames : " , num_frames)

#video = cv2.VideoWriter(OUTPUT_PATH + OUTPUT_FILE ,fourcc, fps, (int(height), int(width)))

パス「/content/drive/My Drive/datasets/」において、
ファイル名「C0003.MP4」の存在が確認できました。

動画の基礎データを取得します... 

<VideoCapture 0x7fe5f19e3f70>
fps     :  59.94005994005994
width  :  1920.0
height :  1080.0
frames :  270.0


##2.5. モデルを作成する。

In [ ]:
import mxnet as mx
from gluoncv.model_zoo import get_model

ctx = mx.cpu()
detector_name = "ssd_512_mobilenet1.0_coco"
detector = get_model(detector_name, pretrained=True, ctx=ctx)


detector.reset_class(classes=['person'], reuse_weights={'person':'person'})
detector.hybridize()

estimator = get_model('simple_pose_resnet18_v1b', pretrained='ccd24037', ctx=ctx)
estimator.hybridize()

/usr/local/lib/python3.6/dist-packages/mxnet/gluon/block.py:1512: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]


##2.6. 画像1枚ずつ姿勢推定をする。

In [ ]:
import gluoncv as gcv
from gluoncv.data.transforms.pose import detector_to_simple_pose, heatmap_to_coord
from gluoncv.utils.viz import cv_plot_image, cv_plot_keypoints

import time
import sys
import numpy as np
from tqdm.autonotebook import tqdm

from gluoncv import utils
import matplotlib.pyplot as plt
import io

In [ ]:
cap = cv2.VideoCapture(INPUT_PATH+INPUT_FILE) #全てのセルを一度に動かす際は、この行は無くても良い。

print("姿勢推定を開始します...")
print()
i=0
for i in tqdm(range(int(num_frames))):
    ret, frame0 = cap.read()
    frame = mx.nd.array(cv2.cvtColor(frame0, cv2.COLOR_BGR2RGB)).astype('uint8')

    x, frame = gcv.data.transforms.presets.ssd.transform_test(frame, short=512, max_size=350)
    x = x.as_in_context(ctx)
    class_IDs, scores, bounding_boxs = detector(x)

    pose_input, upscale_bbox = detector_to_simple_pose(frame, class_IDs, scores, bounding_boxs,
                                                       output_shape=(128, 96), ctx=ctx)
    if len(upscale_bbox) > 0:
        predicted_heatmap = estimator(pose_input)
        pred_coords, confidence = heatmap_to_coord(predicted_heatmap, upscale_bbox) #pred_coordsが座標, confidenceが信頼度を表す。

        #------------------------------------------------------------------------------------------------------------
        #こっから画像に座標を描画する。
        #------------------------------------------------------------------------------------------------------------
        img = utils.viz.plot_keypoints(frame, pred_coords, confidence, class_IDs, 
                                bounding_boxs, scores, box_thresh=1, keypoint_thresh=0.2)
        
        # can't read image, escape
        if img is None:
          print("can't read")
          break
        
        # add
        img.axis('off')
        img=img.get_figure()
        img.savefig(OUTPUT_PATH + "%s.jpg" % (str(i).zfill(4)), transparent = True, bbox_inches = 'tight', pad_inches = 0, dpi=300) 
        plt.close()
        #------------------------------------------------------------------------------------------------------------

    time.sleep(0.01)

    #cv2_imshow(img)
print('姿勢推定が完了しました。')
print()

#video.release()

姿勢推定を開始します...




姿勢推定が完了しました。



##2.7. 画像を結合して動画に変換する。

In [ ]:
# 画像結合
import glob
images = glob.glob(OUTPUT_PATH + "*.jpg")

#---------------------------------------------------------
#globで取得したリストをソートする関数
#---------------------------------------------------------
from natsort import natsorted
images=sorted(images)
#---------------------------------------------------------

print("画像を結合して動画を作成します...")
fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
video = cv2.VideoWriter(OUTPUT_PATH + OUTPUT_FILE, fourcc, fps, (int(width), int(height)))

from tqdm.autonotebook import tqdm
for i in tqdm(range(len(images))):
    img = cv2.resize(cv2.imread(images[i]),(int(width),int(height)))
    video.write(img) 

video.release()
print("%sにおいて、\n%sを作成しました。" % (OUTPUT_PATH, OUTPUT_FILE))

画像を結合して動画を作成します...



/content/drive/My Drive/datasets/PoseEstim_GluonCV_2020-09-17-15-35/において、
PoseEstim_2020-09-17-15-35_from_C0003.MP4を作成しました。


-補足-

ffmpegを使用する場合は、以下をcolabで打つ。

https://qiita.com/croquette0212/items/8ff251d5da77e803c253

もしくは、以下をPC上のシェルで打つ。

`ffmpeg -r 60 -i %04d.jpg -vf scale=trunc(iw/2)*2:trunc(ih/2)*2 -r 24 out.mp4'`

https://qiita.com/genchi-jin/items/90078b6ec751fdacbc9e

##2.8 完成した動画を表示する。

https://stackoverrun.com/ja/q/4910982

In [ ]:
import base64
import io
from IPython.display import HTML

def play_movie(mp4_path):
  video = io.open(mp4_path, 'r+b').read()
  encoded = base64.b64encode(video)
  return HTML(data='''<video width="640" height="480" loop="true" autoplay="true" controls> 
                      <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                      </video>'''.format(encoded.decode('utf-8')))

print("出力画像 : ", OUTPUT_PATH+OUTPUT_FILE)

#play_movie(OUTPUT_PATH+OUTPUT_FILE)

出力画像 :  /content/drive/My Drive/datasets/PoseEstim_GluonCV_2020-09-17-15-35/PoseEstim_2020-09-17-15-35_from_C0003.MP4


In [ ]:
import os
import cv2

if os.path.isfile(OUTPUT_PATH+OUTPUT_FILE) ==True:
  print("パス「%s」において、\nファイル名「%s」の存在が確認できました。\n" % (OUTPUT_PATH, OUTPUT_FILE))
  print("動画の基礎データを取得します... ")
  print()
else:
  print("パス「%s」において、\nファイル名「%s」の存在が確認できません。パスを見直してください。" % (OUTPUT_PATH, OUTPUT_FILE))

cap = cv2.VideoCapture(OUTPUT_PATH+OUTPUT_FILE)
#time.sleep(1)  ### letting the camera autofocus
print(cap)

# encoder(for mp4)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
# output file name, encoder, fps, size(fit to image size)
width= cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)

axes = None
num_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

print("fps     : ", fps)
print("width  : ", width)
print("height : ", height)
print("frames : " , num_frames)

#video = cv2.VideoWriter(OUTPUT_PATH + OUTPUT_FILE ,fourcc, fps, (int(height), int(width)))

パス「/content/drive/My Drive/datasets/PoseEstim_GluonCV_2020-09-17-15-35/」において、
ファイル名「PoseEstim_2020-09-17-15-35_from_C0003.MP4」の存在が確認できました。

動画の基礎データを取得します... 

<VideoCapture 0x7fe60c0dd490>
fps     :  59.94
width  :  1920.0
height :  1080.0
frames :  270.0


##2.9. 不要なデータを削除する。

In [ ]:
from tqdm.autonotebook import tqdm
import os

print('使用済みデータを削除します...')
for i in tqdm(range(len(images))):
    os.remove(images[i])

print()
print('終了')

使用済みデータを削除します...




終了




---



#3. データの操作  "ここから途中です"


In [ ]:
cap = cv2.VideoCapture(INPUT_PATH+INPUT_FILE) #全てのセルを一度に動かす際は、この行は無くても良い。
for i in tqdm(range(int(num_frames))):
    ret, frame0 = cap.read()
    frame = mx.nd.array(cv2.cvtColor(frame0, cv2.COLOR_BGR2RGB)).astype('uint8')

    x, frame = gcv.data.transforms.presets.ssd.transform_test(frame, short=512, max_size=350)
    x = x.as_in_context(ctx)
    class_IDs, scores, bounding_boxs = detector(x)

    pose_input, upscale_bbox = detector_to_simple_pose(frame, class_IDs, scores, bounding_boxs,
                                                       output_shape=(128, 96), ctx=ctx)
    if len(upscale_bbox) > 0:
        predicted_heatmap = estimator(pose_input)
        pred_coords, confidence = heatmap_to_coord(predicted_heatmap, upscale_bbox) #pred_coordsが座標, confidenceが信頼度を表す。

        #------------------------------------------------------------------------------------------------------------
        #こっから座標 (pred_coords) と 信頼度 (confidence) を操作可能な状態へ変換する。
        #------------------------------------------------------------------------------------------------------------
        #mxNetのNDArrayからNumpyへ型を変換する。
        pred_coords_v2 = pred_coords.asnumpy()
        confidence_v2 = confidence.asnumpy()
        
        #2つのshapeを揃える。
        pred_coords_v3=pred_coords_v2[:,:,0]
        confidence_v3=confidence_v2[:,:,0]
        #------------------------------------------------------------------------------------------------------------
    
        #------------------------------------------------------------------------------------------------------------
        #こっからテンソル型のデータを作成。(特に意味はない。とりあえず作ってみる。)
        #------------------------------------------------------------------------------------------------------------
        #(x座標, y座標, 信頼度)の順に列方向(axes=1)へ結合
        Output_PoseEstim=np.concatenate([pred_coords_v3, confidence_v3], 1)

        #print(Output_PoseEstim)
        if i==0:
          Output_PoseEstim_Tensor=Output_PoseEstim
        else:
          Output_PoseEstim_Tensor=np.dstack([Output_PoseEstim_Tensor, Output_PoseEstim])

    time.sleep(0.01) 
print(Output_PoseEstim_Tensor.shape)                          

ValueError: ignored

In [ ]:
print(pred_coords_v3)
print()
print(confidence_v3)
print()

print(Output_PoseEstim)